In [1]:
%load_ext snakeviz

In [2]:
from radd import build, vis, fit, models, utils

In [3]:
m = build.Model(data=redf_store.redata, inits=rebsl_boot_popt, kind='xradd', fit_whole_model=True)


      Model is prepared to fit xradd model (w/ hyp dynamic bias) to average data,
      allowing Drift-Rate to vary across levels of Cond (bsl, pnl)

      What is this... a model for ANTS!? 



In [4]:
from numpy.random import sample as rs
from numpy import array
import numba
analyze = True

In [51]:
m.make_simulator()
simulator = m.simulator

p = simulator.vectorize_params(m.inits)
Pg, Tg = simulator.__update_go_process__(p)
Ps, Ts = simulator.__update_stop_process__(p)

      # a/tr/v Bias: ALL CONDITIONS, ALL SSD
DVg = p['z'] + (simulator.xtb[:,None]*np.cumsum(np.where((rs((simulator.ncond, simulator.ntot, Tg.max())).T<Pg), simulator.dx,-simulator.dx).T, axis=2))
      
      #init_ss = array([[DVc[:simulator.nss, ix] for ix in np.where(Ts<Tg[i], Tg[i]-Ts, 0)] for i, DVc in enumerate(DVg)])
      #DVs = init_ss[:, :, :, None]+np.cumsum(np.where(rs((simulator.nss, Ts.max()))<Ps, simulator.dx, -simulator.dx), axis=1)
      #DVs = self.get_ssbase(Ts,Tg,DVg) + np.cumsum(np.where(rs((self.nss, Ts.max()))<Ps, self.dx, -self.dx), axis=1)

In [46]:
DVg.shape

(2, 10000, 645)

In [126]:
x = rs((2,2,5)) + np.arange(5)
x[:, ::2, :]

array([[[ 0.01713871,  1.67740044,  2.89535477,  3.61765567,  4.99925245]],

       [[ 0.4555041 ,  1.80024762,  2.9307012 ,  3.2789667 ,  4.810573  ]]])

In [133]:
mu = np.cumsum(np.append((DVg, -DVg[:, :, ::-1])), axis=2)

TypeError: append() takes at least 2 arguments (1 given)

In [137]:
np.concatenate(DVg, -DVg[:, :, ::-1])

TypeError: only length-1 arrays can be converted to Python scalars

In [125]:
np.argmax(x>3, axis=2)*.0005

array([[3, 3],
       [3, 3]])

In [106]:
np.unravel_index((x.T>a).T.argmax(), x.shape)

(0, 0, 0)

In [118]:
x.argmax(axis=2)

array([[4, 4],
       [4, 4]])

In [109]:
base = DVg[0,0,0]
DVg = DVg-base
a = p['a']-base
#rt = np.amax((DVg.T>=a).T, axis=2)*.0005
np.unravel_index((x.T>=a).T.argmax(axis=2), DVg.shape)

TypeError: Iterator operand 0 dtype could not be cast from dtype('float64') to dtype('int64') according to the rule 'same_kind'

In [53]:
np.argmax(DVg)

0.22956217500000003

In [43]:
np.argmax(DVg.T>a, axis=2)

0.0

In [46]:
%timeit out = sim(m.inits)

1 loops, best of 3: 499 ms per loop


In [54]:
%timeit out = sim_nojit(m.inits)

1 loops, best of 3: 969 ms per loop


In [64]:
DVs.shape

(2, 5, 5000, 855)

In [68]:
%timeit init_ss.T + DVs.T

1 loops, best of 3: 328 ms per loop


In [70]:
%timeit init_ss[:, :, :, None] + DVs

1 loops, best of 3: 295 ms per loop


In [92]:

dg = lambda Tg, Pg, p: p['z'] + (simulator.get_xtb(Tg, p['xb'])[:,None]*np.cumsum(np.where((rs((2, 10000, Tg.max())).T<Pg), simulator.dx,-simulator.dx).T, axis=2))
ds = lambda Ts, Ps, Tg, DVg: simulator.get_ssbase(Ts,Tg,DVg) + np.cumsum(np.where(rs((simulator.nss, Ts.max()))<Ps, simulator.dx, -simulator.dx), axis=1)

In [97]:
%timeit yhat = sim_nojit(m.inits)

1 loops, best of 3: 1.06 s per loop


In [93]:
def sim_nojit(inits, analyze = True):

      p = simulator.vectorize_params(inits)
      Pg, Tg = simulator.__update_go_process__(p)
      Ps, Ts = simulator.__update_stop_process__(p)
      
      DVg = dg(Tg, Pg, p)
      DVs = ds(Ts, Ps, Tg, DVg)
      
      if analyze:      
            out = simulator.analyze_reactive(DVg, DVs, p)
      else:
            out = [DVg, DVs]
      return out

In [96]:
def sim_nojit(inits, analyze = True):

      p = simulator.vectorize_params(inits)
      Pg, Tg = simulator.__update_go_process__(p)
      Ps, Ts = simulator.__update_stop_process__(p)

      # a/tr/v Bias: ALL CONDITIONS, ALL SSD
      DVg = p['z'] + (simulator.xtb[:,None]*np.cumsum(np.where((rs((2, 10000, Tg.max())).T<Pg), simulator.dx,-simulator.dx).T, axis=2))
      
      #init_ss = array([[DVc[:simulator.nss, ix] for ix in np.where(Ts<Tg[i], Tg[i]-Ts, 0)] for i, DVc in enumerate(DVg)])
      #DVs = init_ss[:, :, :, None]+np.cumsum(np.where(rs((simulator.nss, Ts.max()))<Ps, simulator.dx, -simulator.dx), axis=1)
      DVs = simulator.get_ssbase(Ts,Tg,DVg) + np.cumsum(np.where(rs((simulator.nss, Ts.max()))<Ps, simulator.dx, -simulator.dx), axis=1)

      if analyze:
            
            out = simulator.analyze_reactive(DVg, DVs, p)
      else:
            out = [DVg, DVs]
      return out

In [72]:
m.make_simulator(); simulator=m.simulator

In [83]:
simulator.dx

0.0022360679774997899

In [82]:
Ts.max()

855

In [77]:
%timeit yhat = simulator.sim_fx(m.inits, analyze=True)

1 loops, best of 3: 1.08 s per loop


In [86]:
%timeit yhat = simulator.sim_fx(m.inits, analyze=True)

1 loops, best of 3: 1.06 s per loop


In [76]:
%timeit yhat = simulator.sim_fx(m.inits, analyze=True)

1 loops, best of 3: 1.14 s per loop


In [12]:
%%snakeviz 
yhat = simulator.sim_fx(m.inits, analyze=True)

 
*** Profile stats marshalled to file u'/var/folders/b2/vncnn5695ss7hd1_0fw3s1rr0000gn/T/tmpppFt6T'. 


In [24]:
x = np.arange(200).reshape(4,50)
y = np.random.sample(200).reshape(4,50)

In [28]:
%timeit np.random.sample((5,5,10000))

100 loops, best of 3: 4.12 ms per loop


In [29]:
%timeit np.random.random_sample((5,5,10000))

100 loops, best of 3: 4.19 ms per loop


In [23]:
%timeit np.random.rand(5,5,10000)

100 loops, best of 3: 5.02 ms per loop


In [7]:
%%prun
yhat = simulator.sim_fx(m.inits, analyze=True)

In [8]:
yhat = simulator.sim_fx(m.inits, analyze=True)